In [1]:
# import packages
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import cnmfereview as cr
import config as cfg
import os
from joblib import dump, load

Using TensorFlow backend.
/Users/hassaku/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hassaku/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hassaku/.pyenv/versions/anaconda3-5.0.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16",

In [2]:
MODELDIR = Path('../best_models')

## Load your data

In [3]:
data = cr.Dataset(
    data_paths=cfg.data_paths,
    exp_id=cfg.exp_id,
    img_shape=cfg.img_shape,
    img_crop_size=cfg.img_crop_size,
    max_trace=cfg.max_trace_len,
    )

No preprocessing on spatial data
File ../data/cr_tutorialA_cropped.npy already exists and has been loaded instead.
No preprocessing on trace data.                   ../data/cr_tutorialCraw_normalized.npy already                   exists and has been loaded instead.
Successfully loaded data.


In [ ]:
x_train, x_test, y_train, y_test = data.split_training_test_data(
    test_split=.20,
    seed=10
)

_________________
**NOTE: Remove the next cell when training your own models.** This step uses fewer ROIs (only ~3000 instead of 11 000) in the tutorial dataset to speed up computation in the tutorial. Do not do this when you are training your own data. You want to use as many data samples as possible to get the best results in practice. 

In [ ]:
# remove or comment out this cell when using on your own data
from sklearn.model_selection import train_test_split
x_train, _, y_train, _ = train_test_split(x_train, y_train, test_size=0.75)

____________________________

In [ ]:
print(f"Number of samples in training set: {x_train.shape[0]}") 
print(f"Number of samples in test set: {x_test.shape[0]}")

# Train the saved models on your data

# TPOT Classifier

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, accuracy_score
from sklearn.svm import LinearSVC

# this was the final TPOT exported pipeline that acheived the highest F1 score
tpot_model = LinearSVC(C=0.1, dual=False, loss="squared_hinge", penalty="l1", tol=0.1)
tpot_model.fit(x_train, y_train)

In [ ]:
tpot_y_pred = tpot_model.predict(x_train)
print("Accuracy:", accuracy_score(y_train, tpot_y_pred))
print("f1:", f1_score(y_train, tpot_y_pred))
dump(tpot_model, MODELDIR / f'{cfg.exp_id}_tpot.joblib') 

Save a copy of the model finetuned on your data to use again in the future to predict without having to retrain.

# AutoSklearn Classifier

In [4]:
import autosklearn
import sklearn
# load the AutoSklearn ensemble object
askl = load(MODELDIR / 'cr_tutorial_askl.joblib')
askl.refit(x_train, y_train)

KeyboardInterrupt: 

In [ ]:
results_automl = askl.predict(x_test)
print("Accuracy:", accuracy_score(y_test, results_automl))
print("f1:", f1_score(y_test, results_automl))
dump(askl, MODELDIR / f'{cfg.exp_id}_askl.joblib')

# Deep Classifier

In [5]:
data.spatial.shape, data.trace.shape, data.targets.shape
x_train, x_test, y_train, y_test = data.split_training_test_data(
    test_split=.20, seed=10, for_deep=True)
model = cr.get_model(with_spatial=True, use_cnn_for_trace=True, frame=500, crop_size=80)

Training and test data loaded


In [ ]:
callbacks = []
# callbacks.append(keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode='auto'))

fit = model.fit(x_train, y_train,
          batch_size=16,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test),
          callbacks=callbacks)

Instructions for updating:
Use tf.cast instead.
Train on 11603 samples, validate on 2901 samples
Epoch 1/10
11603/11603 [==============================] - 150s 13ms/step - loss: 0.3724 - acc: 0.8442 - val_loss: 0.2581 - val_acc: 0.8952
Epoch 2/10
11603/11603 [==============================] - 156s 13ms/step - loss: 0.2983 - acc: 0.8797 - val_loss: 0.2461 - val_acc: 0.8993
Epoch 3/10
 3424/11603 [=======>......................] - ETA: 1:35 - loss: 0.2882 - acc: 0.8823

# Apply classifiers to unlabeled data

In [ ]:
askl = load(MODELDIR / f'{cfg.exp_id}_askl.joblib');
tpot_model = load(MODELDIR / f'{cfg.exp_id}_tpot.joblib')
cfg.img_shape

In [ ]:
unseen_data = cr.UnlabeledDataset(
    mat_file='../data/unlabeled_rois_DM298.mat',
    img_shape={'x': 284, 'y': 231},
    img_crop_size=cfg.img_crop_size,
    max_trace=cfg.max_trace_len)

In [ ]:
cfg.img_shape, cfg.img_crop_size, cfg.max_trace_len

In [ ]:
pred_askl = askl.predict(unseen_data.combined)
pred_tpot = tpot_model.predict(unseen_data.combined)

In [ ]:
# preview the ROIs labeled by askl as "positives"
positive_askl = np.where(pred_askl==1)[0]
# limit to only show 10 at once, you can play around with this of course
cr.plot_rois(unseen_data, positive_askl[:10])

In [ ]:
# preview the ROIs labeled by askl as "negatives"
negative_askl = np.where(pred_askl==0)[0]
# limit to only show 10 at once, you can play around with this of course
cr.plot_rois(unseen_data, negative_askl[:10])

In [ ]:
gt_label = [1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
accuracy_score(gt_label, pred_askl), f1_score(gt_label, pred_askl)

In [ ]:
# preview the ROIs labeled by TPOT as "negatives"
cr.plot_rois(unseen_data, np.where(pred_tpot==0)[0][:10])

# Apply reviews

In [ ]:
unseen_data.apply_labels(pred_askl)

In [ ]:
# load the file to check the results
from scipy.io import loadmat, savemat

labeled_data = loadmat('../data/unlabeled_rois_automl.mat')